# RLlib Sample Application: CartPole-v0

We were briefly introduced to the _CartPole_ example and the OpenAI gym `CartPole-v0` environment ([gym.openai.com/envs/CartPole-v0/](https://gym.openai.com/envs/CartPole-v0/)) in the [introduction](01-Introduction-to-Reinforcement-Learning.ipynb). This lesson uses [RLlib](https://ray.readthedocs.io/en/latest/rllib.html) to train a policy for _CartPole_.

Even though this is a relatively simple and quick example to run, its results can be understood quite visually.

For more background about this problem, see:

* ["Neuronlike Adaptive Elements That Can Solve Difficult Learning Control Problem"](https://ieeexplore.ieee.org/document/6313077), AG Barto, RS Sutton, and CW Anderson, *IEEE Transactions on Systems, Man, and Cybernetics* (1983)
* ["Cartpole - Introduction to Reinforcement Learning (DQN - Deep Q-Learning)"](https://towardsdatascience.com/cartpole-introduction-to-reinforcement-learning-ed0eb5b58288), [Greg Surma](https://twitter.com/GSurma)

First, import Ray and the PPO module in RLlib, then start Ray.

In [1]:
import ray
import ray.rllib.agents.ppo as ppo

In [2]:
ray.init(ignore_reinit_error=True)

2020-05-01 14:21:35,492	INFO resource_spec.py:212 -- Starting Ray with 4.98 GiB memory available for workers and up to 2.49 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-05-01 14:21:35,847	INFO services.py:1148 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '192.168.1.149',
 'redis_address': '192.168.1.149:40229',
 'object_store_address': '/tmp/ray/session_2020-05-01_14-21-35_478437_32536/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-05-01_14-21-35_478437_32536/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-05-01_14-21-35_478437_32536'}

The Ray Dashboard is useful for monitoring Ray:

In [3]:
print(f'Dashboard URL: http://{ray.get_webui_url()}')

Dashboard URL: http://localhost:8265


Next we'll train an RLlib policy with the [`CartPole-v0` environment](https://gym.openai.com/envs/CartPole-v0/).

By default, training runs for `10` iterations. Increase the `N_ITER` setting if you want to train longer and see the resulting rewards improve.
Also note that *checkpoints* get saved after each iteration into the `/tmp/ppo/cart` directory.

> **Note:** If you prefer to use a different directory root than `/tmp`, change it in the next cell **and** in the `rllib rollout` command below.

In [5]:
SELECT_ENV = "CartPole-v0"
N_ITER = 10

config = ppo.DEFAULT_CONFIG.copy()
config["log_level"] = "WARN"

reward_history = []

agent = ppo.PPOTrainer(config, env=SELECT_ENV)

for _ in range(N_ITER):
    result = agent.train()
    print(result)

    max_reward = result["episode_reward_max"]
    reward_history.append(max_reward)

    file_name = agent.save('/tmp/ppo/cart')
    print(f'\ncheckpoint saved to {file_name}')

2020-05-01 14:22:04,507	INFO trainer.py:428 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2020-05-01 14:22:04,531	INFO trainer.py:585 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2020-05-01 14:22:07,483	INFO trainable.py:217 -- Getting current IP.
2020-05-01 14:22:07,484	WARNING util.py:37 -- Install gputil for GPU system monitoring.


{'episode_reward_max': 63.0, 'episode_reward_min': 9.0, 'episode_reward_mean': 22.497175141242938, 'episode_len_mean': 22.497175141242938, 'episodes_this_iter': 177, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [27.0, 13.0, 10.0, 17.0, 11.0, 24.0, 18.0, 17.0, 24.0, 17.0, 13.0, 26.0, 19.0, 32.0, 29.0, 11.0, 21.0, 15.0, 56.0, 20.0, 21.0, 26.0, 22.0, 12.0, 27.0, 35.0, 28.0, 19.0, 17.0, 34.0, 12.0, 11.0, 14.0, 13.0, 25.0, 15.0, 20.0, 23.0, 21.0, 37.0, 14.0, 16.0, 13.0, 25.0, 18.0, 58.0, 11.0, 22.0, 43.0, 14.0, 24.0, 15.0, 20.0, 11.0, 45.0, 22.0, 16.0, 13.0, 63.0, 9.0, 25.0, 62.0, 20.0, 26.0, 31.0, 20.0, 20.0, 12.0, 11.0, 24.0, 24.0, 26.0, 25.0, 22.0, 34.0, 31.0, 11.0, 13.0, 30.0, 31.0, 15.0, 26.0, 14.0, 13.0, 17.0, 15.0, 10.0, 34.0, 13.0, 17.0, 12.0, 22.0, 24.0, 59.0, 14.0, 9.0, 32.0, 28.0, 25.0, 29.0, 11.0, 15.0, 12.0, 16.0, 30.0, 19.0, 11.0, 15.0, 14.0, 18.0, 13.0, 23.0, 13.0, 22.0, 23.0, 13.0, 13.0, 20

In [6]:
print(reward_history)

[63.0, 135.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0]


Do the episode rewards increase after multiple iterations?
That shows how the policy is improving.

Also, print out the policy and model to see the results of training in detail…

In [7]:
import pprint

policy = agent.get_policy()
model = policy.model

pprint.pprint(model.variables())
pprint.pprint(model.value_function())

print(model.base_model.summary())

[<tf.Variable 'default_policy/fc_1/kernel:0' shape=(4, 256) dtype=float32>,
 <tf.Variable 'default_policy/fc_1/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_1/kernel:0' shape=(4, 256) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_1/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'default_policy/fc_2/kernel:0' shape=(256, 256) dtype=float32>,
 <tf.Variable 'default_policy/fc_2/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_2/kernel:0' shape=(256, 256) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_2/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'default_policy/fc_out/kernel:0' shape=(256, 2) dtype=float32>,
 <tf.Variable 'default_policy/fc_out/bias:0' shape=(2,) dtype=float32>,
 <tf.Variable 'default_policy/value_out/kernel:0' shape=(256, 1) dtype=float32>,
 <tf.Variable 'default_policy/value_out/bias:0' shape=(1,) dtype=float32>]
<tf.Tensor 'Reshape:0' shape=(?,) dtype=float32>
Model: "model"
__________

Next we'll use the [`rollout` script](https://ray.readthedocs.io/en/latest/rllib-training.html#evaluating-trained-policies), using the `rllib rollout` command line, to evaluate the trained policy.

This visualizes the "cartpole" agent operating within the simulation: moving the cart left or right to avoid having the pole fall over.

In [8]:
! rllib rollout \
    /tmp/ppo/cart/checkpoint_2/checkpoint-2 \
    --config "{\"env\": \"CartPole-v0\"}" --run PPO \
    --steps 2000

2020-05-01 14:23:47,357	INFO resource_spec.py:212 -- Starting Ray with 4.39 GiB memory available for workers and up to 2.2 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-05-01 14:23:47,686	INFO services.py:1148 -- View the Ray dashboard at localhost:8266
2020-05-01 14:23:48,365	INFO trainer.py:428 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2020-05-01 14:23:48,420	INFO trainer.py:585 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2020-05-01 14:23:52,876	INFO trainable.py:217 -- Getting current IP.
2020-05-01 14:23:52,876	WARNING util.py:37 -- Install gputil for GPU system monitoring.
2020-05-01 14:23:52,970	INFO trainable.py:217 -- Getting current IP.
2020-05-01 14:23:52,970	INFO trainable.py:423 -- Restored on 192.168.1.149 from checkpoint: /tmp/ppo/cart/checkpoint_2/checkpoint-2
2020-05-01 14:23:52,970	INFO trainab

The rollout uses the second saved checkpoint, evaluated through `2000` steps.
Modify the path to view other checkpoints.

Finally, launch [TensorBoard](https://ray.readthedocs.io/en/latest/rllib-training.html#getting-started) then follow the instructions (for example, click the URL link or copy and paste it into a browser) to visualize key metrics from training with RLlib…

TODO: explain what to do with Tensorboard

In [ ]:
!tensorboard --logdir=~/ray_results/

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.1.1 at http://localhost:6007/ (Press CTRL+C to quit)
W0501 14:25:27.537329 123145495740416 core_plugin.py:200] Unable to get first event timestamp for run PPO_CartPole-v0_2020-03-26_15-11-39yyf7qw4j: No event timestamp could be found
W0501 14:25:27.539767 123145495740416 core_plugin.py:200] Unable to get first event timestamp for run PPO_CartPole-v0_2020-03-26_15-21-05wb9qng9p: No event timestamp could be found
W0501 14:25:27.542510 123145495740416 core_plugin.py:200] Unable to get first event timestamp for run PPO_CartPole-v0_2020-03-26_15-43-39ai8xvs0i: No event timestamp could be found
W0501 14:25:27.557257 123145495740416 core_plugin.py:200] Unable to get first event timestamp for run PPO_CartPole-v0_2020-05-01_10-29-417oh9pqog: No event timestamp could be found
W0501 14:25:27.558618 123145495740416 core_plugin.py:200] Unable to get first event timestamp for run PPO_CartPole-v0_

## Exercise 1

TODO

Next, go through any of the following lessons:

* [04a: Application: Mountain Car](04a-Application-Mountain-Car.ipynb) -- Based on the `MountainCar-v0` environment from OpenAI Gym.
* [04b: Application: Taxi](04b-Application-Taxi.ipynb) -- Based on the `Taxi-v3` environment from OpenAI Gym.
* [04c: Application: Frozen Lake](04c-Application-Frozen-Lake.ipynb) -- Based on the `FrozenLake-v0` environment from OpenAI Gym.